In [1]:
!pip install tensorflow

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from collections import Counter

# Загрузка данных
final_df = pd.read_csv('dataset.csv')  # Замените на путь к вашему датасету
user_tracks_df = pd.read_csv('favorite_tracks.csv')  # Датафрейм с любимыми треками пользователей


In [37]:
# Создаем целевую переменную
def create_target(user_tracks_df, final_df):
    target = []
    for track_id in final_df['track_id']:
        is_favorite = any(track_id in user_tracks_df[user_tracks_df['user'] == user_id].values[0][1:] for user_id in user_tracks_df['user'])
        target.append(1 if is_favorite else 0)
    return target

final_df['liked'] = create_target(user_tracks_df, final_df)


In [38]:
# Выбор признаков и целевой переменной
features = ['artist_id', 'popularity', 'danceability', 'energy', 'key', 'loudness',
            'mode', 'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
X = final_df[features]
y = final_df['liked']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [39]:
# Определение нейросетевой модели
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.5088 - loss: 0.6995 - val_accuracy: 0.8864 - val_loss: 0.5599
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7881 - loss: 0.5889 - val_accuracy: 0.8864 - val_loss: 0.4549
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8018 - loss: 0.5250 - val_accuracy: 0.8864 - val_loss: 0.4137
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7548 - loss: 0.5571 - val_accuracy: 0.8864 - val_loss: 0.4174
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7651 - loss: 0.5670 - val_accuracy: 0.8864 - val_loss: 0.4265
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8009 - loss: 0.5082 - val_accuracy: 0.8864 - val_loss: 0.4175
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7770 - loss: 0.5113 - val_accuracy: 0.8864 - val_loss: 0.4152
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8057 - loss: 0.4607 - val_accuracy: 0.8864 - val_loss: 0.4094

In [42]:
from collections import Counter

def recommend_songs_with_entropy_penalty(model, user_tracks_df, final_df, scaler, penalty_weight=0.5):
    recommendations = {}
    track_counts = Counter()  # Для подсчета популярности каждого трека в рекомендациях

    for user_id in user_tracks_df['user']:
        # Прогноз вероятности "лайка" каждого трека пользователем
        user_tracks = user_tracks_df[user_tracks_df['user'] == user_id].values[0][1:]
        candidate_tracks = final_df[~final_df['track_id'].isin(user_tracks)]

        candidate_features = scaler.transform(candidate_tracks[features])
        predictions = model.predict(candidate_features).flatten()

        # Применение энтропийного штрафа
        # Конвертируем track_counts.values() в массив NumPy для корректного сложения
        penalty = penalty_weight * np.log(1 + np.array([track_counts[track] for track in candidate_tracks['track_id']]))
        candidate_tracks['score'] = predictions - penalty

        # Сортировка и выбор топ-3 рекомендаций
        top_recs = candidate_tracks.sort_values(by='score', ascending=False).head(3)['track_id'].values
        recommendations[user_id] = top_recs

        # Обновление счётчика популярности треков
        track_counts.update(top_recs)

    return recommendations


In [43]:
# Генерация рекомендаций
predictions = recommend_songs_with_entropy_penalty(model, user_tracks_df, final_df, scaler)

# Создание таблицы рекомендаций
recommendations_df = pd.DataFrame({
    'user': list(predictions.keys()),
    'rec_1': [recs[0] if len(recs) > 0 else None for recs in predictions.values()],
    'rec_2': [recs[1] if len(recs) > 1 else None for recs in predictions.values()],
    'rec_3': [recs[2] if len(recs) > 2 else None for recs in predictions.values()],
})

# Сохраняем рекомендации в CSV файл
recommendations_df.to_csv('user_recommendations_Keras.csv', index=False)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty
<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
 1/17 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

...

<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty
<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty
<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty
<ipython-input-42-ae691f1ea800>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_tracks['score'] = predictions - penalty
